# Fixing the "er_[0-9]{1}_params" Simplex-Index Problem
To get the Rev scripts working to correspond with how the Simplex class cannot be indexed element-wise, we will write a few functions, then voila! Excuse all my variable names... The ones that I have named otherwise cartoonishly were simply not important enough to be named, but important enough to be saved as their own variable.

In [1]:
# LIBRARIES
import re

I need regex to handle finding the variable names.

# 1. For A Single Script
Like always, start with any old script as a model to explore how the functions should be written. I chose one of the honky-tonk datasets that gives me a problem all the time.

In [13]:
# LOAD IN SCRIPT
aic_45 = "/Users/treehouse5/Edie/RevBayes (9:29)/45/45_aic.Rev"

In [14]:
with open (aic_45, "r") as this_file:
    data = this_file.read()

In [16]:
print data

##########################
# Read the sequence data #
##########################

data_ND2_codon_1 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_1.nex")
data_ND2_codon_2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_2.nex")
data_ND2_codon_3 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_3.nex")
data_ND3_codon_1 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_1.nex")
data_ND3_codon_2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_2.nex")
data_ND3_codon_3 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_3.nex")
data_TGFB2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/TGFB2.nex")
taxa = data_ND2_codon_1.taxa()
num_species = data_ND2_codon_1.ntaxa()
num_branches = 2 * num_species - 2

move_index = 1

#############################################
# Define the tree and associated parameters #
#############################################

root_height <- 1
d

### 1a. Exploring Script Pieces
I'm going to cut up the script line by line to make a generalisable function that can find where the er_[0-9]{1}_params are.

In [76]:
pieces = data.split("\n")

In [77]:
len(pieces)

224

In [82]:
for this_line in range(len(pieces)):
    if "# Relative rates." in pieces[this_line]:
        start_ind = this_line
    if "# Stationary frequencies." in pieces[this_line]:
        end_ind = this_line

In [83]:
start_ind, end_ind

(46, 66)

I plan to replace this entire area (the area spanning from line 46 to line 66) with new text. Since I cut these pieces up, I am going to put them back together because I found the section lines that I want to work with.

In [90]:
blah = "\n".join(pieces[48:66])

In [91]:
print blah

er_1_params ~ dnDirichlet(v(1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_1_params,alpha=10.0,weight=0.5)
er_1 := simplex(er_1_params[1],er_1_params[2],er_1_params[3],er_1_params[4],er_1_params[2],er_1_params[5])

er_2 ~ dnDirichlet(v(1,1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_2,alpha=10.0,weight=0.5)

er_3 ~ dnDirichlet(v(1,1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_3,alpha=10.0,weight=0.5)

er_4_params ~ dnDirichlet(v(1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_4_params,alpha=10.0,weight=0.5)
er_4 := simplex(er_4_params[1],er_4_params[2],er_4_params[3],er_4_params[3],er_4_params[4],er_4_params[1])

er_5_params ~ dnDirichlet(v(1,1,1))
moves[move_index++] = mvSimplexElementScale(er_5_params,alpha=10.0,weight=0.5)
er_5 := simplex(er_5_params[1],er_5_params[2],er_5_params[1],er_5_params[1],er_5_params[3],er_5_params[1])



Now, I'm going to buckle down on just the area that I wanted to fix. I'm going to split it on two spaces instead of just one space because I know the format of the text file. This will allow me to look at each of the er_[0-9]{1}_params and edit the text as necessary.

In [96]:
rawr = blah.split("\n\n")

In [97]:
len(rawr)

5

This is what one of those "rawr" elements looks like.

In [108]:
print rawr[4]

er_5_params ~ dnDirichlet(v(1,1,1))
moves[move_index++] = mvSimplexElementScale(er_5_params,alpha=10.0,weight=0.5)
er_5 := simplex(er_5_params[1],er_5_params[2],er_5_params[1],er_5_params[1],er_5_params[3],er_5_params[1])



Now, I'm going to write a for loop which will identify which elements of "rawr" need a-fixin'.

In [109]:
need_indices = []
for this_line in range(len(rawr)):
    if "simplex" in rawr[this_line]:
        need_indices.append(this_line)

In [110]:
need_indices

[0, 3, 4]

I'm going to check that the 0th element of rawr actually does need a-fixin'. And it does.

In [128]:
print rawr[0]

er_1_params ~ dnDirichlet(v(1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_1_params,alpha=10.0,weight=0.5)
er_1 := simplex(er_1_params[1],er_1_params[2],er_1_params[3],er_1_params[4],er_1_params[2],er_1_params[5])


So now I'm going to split tis 0th element (and all the rest of the elements, later on) by line in order to do the hacky thing to take care of the simplex-index problem.

In [129]:
listed_rawr_0 = rawr[0].split("\n")

Here, I'm going to put that regex into use... I want to do the renames that Mike did but automagically.

In [151]:
m = re.search("er_[0-9]{1}_params", listed_rawr_0[0])

In [152]:
m.group(0)[:-5]

'er_1_p'

In [132]:
m.group(0)

'er_1_params'

And now I'm going to fix the first two instances of the variable name of er_1_params to change them to er_1_p, but on the full 0th element of "rawr" -- if you remember what that is still.

In [133]:
# CHANGE THE FIRST TWO "PARAMS" ONLY
fix1 = rawr[0].replace(m.group(0), m.group(0)[:-5], 2)

In [134]:
print fix1

er_1_p ~ dnDirichlet(v(1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_1_p,alpha=10.0,weight=0.5)
er_1 := simplex(er_1_params[1],er_1_params[2],er_1_params[3],er_1_params[4],er_1_params[2],er_1_params[5])


Now, I'm going to split the lines up again in order to add that new line to convert the simplex into the real-positive-value class.

In [141]:
fix1lines = fix1.split("\n")

In [144]:
new_line = m.group(0)+" := abs("+m.group(0)[:-5]+")"

In [145]:
print new_line

er_1_params := abs(er_1_p)


Lastly, I'm going to piece them together. Luckily for us in this case, we know that all of the "error" lines are in the same structure so we can do a lot of yummy hard coding.

In [147]:
new_block = "\n".join([fix1lines[0], fix1lines[1], new_line, fix1lines[2]])

In [148]:
print new_block

er_1_p ~ dnDirichlet(v(1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_1_p,alpha=10.0,weight=0.5)
er_1_params := abs(er_1_p)
er_1 := simplex(er_1_params[1],er_1_params[2],er_1_params[3],er_1_params[4],er_1_params[2],er_1_params[5])


### 1b. Abstraction to Go Through Code
In the following functions, I am taking the code from above in "1a" to generalize it into several small functions. I'll put these in a main function later.

In [2]:
def find_fix_area(script_path, ind=False):
    with open (script_path, "r") as this_file:
        data = this_file.read()
    
    # NARROW DOWN TEXT
    pieces = data.split("\n")
    
    for this_line in range(len(pieces)):
        if "# Relative rates." in pieces[this_line]:
            start_ind = this_line
        if "# Stationary frequencies." in pieces[this_line]:
            end_ind = this_line
    
    fix_area = "\n".join(pieces[start_ind+2:end_ind])
    
    if ind==True:
        return([start_ind+2, end_ind])
    else:
        return(fix_area)

In [3]:
with open (aic_45, "r") as this_file:
    data = this_file.read()

# NARROW DOWN TEXT
pieces = data.split("\n")

for this_line in range(len(pieces)):
    if "# Relative rates." in pieces[this_line]:
        start_ind = this_line
    if "# Stationary frequencies." in pieces[this_line]:
        end_ind = this_line    

NameError: name 'aic_45' is not defined

In [399]:
aic_45

'/Users/treehouse5/Edie/RevBayes (9:29)/45/45_aic.Rev'

In [400]:
pieces

['##########################',
 '# Read the sequence data #',
 '##########################',
 '',
 'data_ND2_codon_1 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_1.nex")',
 'data_ND2_codon_2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_2.nex")',
 'data_ND2_codon_3 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_3.nex")',
 'data_ND3_codon_1 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_1.nex")',
 'data_ND3_codon_2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_2.nex")',
 'data_ND3_codon_3 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_3.nex")',
 'data_TGFB2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/TGFB2.nex")',
 'taxa = data_ND2_codon_1.taxa()',
 'num_species = data_ND2_codon_1.ntaxa()',
 'num_branches = 2 * num_species - 2',
 '',
 'move_index = 1',
 '',
 '#############################################',
 '# Define the tree and associated para

In [401]:
start_ind

46

In [402]:
end_ind

66

In [394]:
fix_area = "\n".join(pieces[start_ind+2:end_ind])

In [395]:
fix_area

''

In [389]:
print find_fix_area(aic_45)

UnboundLocalError: local variable 'start_ind' referenced before assignment

In [4]:
def fix_single_param(param_area):
#     # SPLIT ON RETURNS
#     param_area_lines = param_area.split("\n")
    
    # GET VARIABLE NAMES
    m = re.search("er_[0-9]{1,2}_params", param_area) # FIRST LINE HAS NAME DESIRED
    first_name = m.group(0)[:-5]
    last_name = m.group(0)
    
    # CHANGE THE FIRST TWO "PARAMS" ONLY
    fix1 = param_area.replace(m.group(0), m.group(0)[:-5], 2)
    
    # SPLIT TO PREPARE TO ADD NEW LINE
    fix1lines = fix1.split("\n")
    
    # CREATE NEW LINE
    new_line = m.group(0)+" := abs("+m.group(0)[:-5]+")"
    
    # CREATE NEW CODE BLOCK
    new_block = "\n".join([fix1lines[0], fix1lines[1], new_line, fix1lines[2]])
    
    return(new_block)

In [156]:
# print fix_single_param(rawr[0])

er_1_p ~ dnDirichlet(v(1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_1_p,alpha=10.0,weight=0.5)
er_1_params := abs(er_1_p)
er_1 := simplex(er_1_params[1],er_1_params[2],er_1_params[3],er_1_params[4],er_1_params[2],er_1_params[5])


In [353]:
print fix_single_param(rawr[3])

er_4_p ~ dnDirichlet(v(1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_4_p,alpha=10.0,weight=0.5)
er_4_params := abs(er_4_p)
er_4 := simplex(er_4_params[1],er_4_params[2],er_4_params[3],er_4_params[3],er_4_params[4],er_4_params[1])


In [354]:
print fix_single_param(rawr[4])

er_5_p ~ dnDirichlet(v(1,1,1))
moves[move_index++] = mvSimplexElementScale(er_5_p,alpha=10.0,weight=0.5)
er_5_params := abs(er_5_p)
er_5 := simplex(er_5_params[1],er_5_params[2],er_5_params[1],er_5_params[1],er_5_params[3],er_5_params[1])


In [5]:
def fix_the_area(code_string):
    # SPLIT ON DOUBLE RETURNS
    param_pieces = code_string.split("\n\n")
    
    # FIND LINE INDICES WHERE CHANGE NEEDED
    need_indices = []
    for this_line in range(len(param_pieces)):
        if "simplex" in param_pieces[this_line]:
            need_indices.append(this_line)
    
    # FIX THOSE LINES
    fixes = [fix_single_param(param_pieces[i]) for i in need_indices]
    
    # RENOVATE OLD FIX AREA
    change_or_not = [0]*len(param_pieces)
    for i in range(len(param_pieces)):
        if i in need_indices:
            change_or_not[i] += 1
    
    new_block = []
    change_ind = 0
    for i in range(len(change_or_not)):
        if change_or_not[i] == 1:
            new_block.append(fixes[change_ind])
            change_ind += 1
        if change_or_not[i] == 0:
            new_block.append(param_pieces[i])
    
    return("\n\n".join(new_block))

In [378]:
print fix_the_area(find_fix_area(aic_45))

er_1_p ~ dnDirichlet(v(1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_1_p,alpha=10.0,weight=0.5)
er_1_params := abs(er_1_p)
er_1 := simplex(er_1_params[1],er_1_params[2],er_1_params[3],er_1_params[4],er_1_params[2],er_1_params[5])

er_2 ~ dnDirichlet(v(1,1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_2,alpha=10.0,weight=0.5)

er_3 ~ dnDirichlet(v(1,1,1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_3,alpha=10.0,weight=0.5)

er_4_p ~ dnDirichlet(v(1,1,1,1))
moves[move_index++] = mvSimplexElementScale(er_4_p,alpha=10.0,weight=0.5)
er_4_params := abs(er_4_p)
er_4 := simplex(er_4_params[1],er_4_params[2],er_4_params[3],er_4_params[3],er_4_params[4],er_4_params[1])

er_5_p ~ dnDirichlet(v(1,1,1))
moves[move_index++] = mvSimplexElementScale(er_5_p,alpha=10.0,weight=0.5)
er_5_params := abs(er_5_p)
er_5 := simplex(er_5_params[1],er_5_params[2],er_5_params[1],er_5_params[1],er_5_params[3],er_5_params[1])


### 1c. Main Function
This will put all of the above functions together to create one wrap-around function that will do the whole job.

In [6]:
def main(script_path, write=False):
    # READ FILE AS STRING
    with open (script_path, "r") as this_file:
        data = this_file.read()
    
    # CUT IT UP
    data_lines = data.split("\n")
    indices = find_fix_area(script_path, ind=True)
    start_ind, end_ind = indices[0], indices[1]
    
    # SUBFIXES
    fix_area = find_fix_area(script_path)
    updated_area = fix_the_area(fix_area)
    
    # PUT TOGETHER AGAIN
    humpty_dumpty = "\n".join(data_lines[:start_ind]) + "\n" + updated_area + "\n\n" + "\n".join(data_lines[end_ind:])
    
    if write==True:
        f = open(script_path, "w")
        f.write(humpty_dumpty)
        f.close()
        return("File written to disk for: "+str(script_path.split("/")[-1]))
    
    else:
        return(humpy_dumpty)

In [379]:
print main(aic_45)

##########################
# Read the sequence data #
##########################

data_ND2_codon_1 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_1.nex")
data_ND2_codon_2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_2.nex")
data_ND2_codon_3 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND2_codon_3.nex")
data_ND3_codon_1 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_1.nex")
data_ND3_codon_2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_2.nex")
data_ND3_codon_3 = readDiscreteCharacterData("data/subsets_no_outgroups/45/ND3_codon_3.nex")
data_TGFB2 = readDiscreteCharacterData("data/subsets_no_outgroups/45/TGFB2.nex")
taxa = data_ND2_codon_1.taxa()
num_species = data_ND2_codon_1.ntaxa()
num_branches = 2 * num_species - 2

move_index = 1

#############################################
# Define the tree and associated parameters #
#############################################

root_height <- 1
d

And it did the whole job! Yee. Now, onto doing it for all of the scripts... How fun.

# 2. For All Scripts in Directory

### 2a. biohazardCleanUp Functions
I save a lot of my frequently used functions under a repository called biohazardCleanUp
<br>
<b>Link</b>: https://github.com/palautatan/biohazardCleanUp/

In [10]:
# REQUIREMENTS
from os import listdir

# LISTDIR_NH FUNCTION
def listdir_nh(path):
    '''
    Input: Path of directory (string)
    Output: Non-hidden files within given directory (list of strings)
    '''
    files = listdir(path)
    parsed = [file for file in files if not file.startswith(".")]
    return parsed

### 2b. Listing My Directory
I'm going to recursively use listdir_nh on my directory.

__I AM DOING REVBAYES 2 FOLDER HERE, CHANGE BACK TO REVABYES FOLDER __

In [11]:
# MY PARENT DIRECTORY
padre = "/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/"

In [12]:
# I AM LISTING ALL FOLDERS IN MY PARENT DIRECTORY
folders = [listdir_nh(padre)][0] # I NEED TO FIX MY BHCU FUNCTION

In [13]:
# PREVIEW OF FIRST FIVE FOLDERS
folders[:5]

['117', '196_8', '21', '28', '45']

In [14]:
# NOW I AM LISTING ALL THE FILES
files_in_folders = [listdir_nh(padre+folder) for folder in folders]

In [15]:
# PREVIEW OF FIRST TWO DATASETS
files_in_folders[:2]

[['117_aic.Rev',
  '117_all_apart.Rev',
  '117_bic.Rev',
  '117_gene.Rev',
  '117_together.Rev'],
 ['196_8_aic.Rev',
  '196_8_all_apart.Rev',
  '196_8_bic.Rev',
  '196_8_gene.Rev',
  '196_8_together.Rev']]

In [16]:
# AND NOW I AM MAKING A LIST OF ALL THE PATHS TO THESE SUBDIRECTORIES
all_paths = [[padre+folders[i]+"/"+x for x in files_in_folders[i]] for i in range(len(folders))]

In [17]:
# PREVIEW
all_paths[:3]

[['/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_aic.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_all_apart.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_bic.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_gene.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_together.Rev'],
 ['/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/196_8/196_8_aic.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/196_8/196_8_all_apart.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/196_8/196_8_bic.Rev',
  '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBaye

In [18]:
# AND NOW TO MAKE ONE LIST
one_list_of_paths = [item for sublist in all_paths for item in sublist]

In [19]:
one_list_of_paths[0:20:2]

['/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_aic.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_bic.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/117/117_together.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/196_8/196_8_all_apart.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/196_8/196_8_gene.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/21/21_aic.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/21/21_bic.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/21/21_together.Rev',
 '/Users/treehouse5/Dropbox/effect_of_partitioning/step_2_shakedown_runs/scripts/RevBayes2/28/28_all_apart.Re

### 2c. "Run It" -Chris Brown

In [20]:
[main(x, True) for x in one_list_of_paths]

['File written to disk for: 117_aic.Rev',
 'File written to disk for: 117_all_apart.Rev',
 'File written to disk for: 117_bic.Rev',
 'File written to disk for: 117_gene.Rev',
 'File written to disk for: 117_together.Rev',
 'File written to disk for: 196_8_aic.Rev',
 'File written to disk for: 196_8_all_apart.Rev',
 'File written to disk for: 196_8_bic.Rev',
 'File written to disk for: 196_8_gene.Rev',
 'File written to disk for: 196_8_together.Rev',
 'File written to disk for: 21_aic.Rev',
 'File written to disk for: 21_all_apart.Rev',
 'File written to disk for: 21_bic.Rev',
 'File written to disk for: 21_gene.Rev',
 'File written to disk for: 21_together.Rev',
 'File written to disk for: 28_aic.Rev',
 'File written to disk for: 28_all_apart.Rev',
 'File written to disk for: 28_bic.Rev',
 'File written to disk for: 28_gene.Rev',
 'File written to disk for: 28_together.Rev',
 'File written to disk for: 45_aic.Rev',
 'File written to disk for: 45_all_apart.Rev',
 'File written to disk f

Voila.